**Importing Libraries**

In [25]:
%matplotlib inline 
%pip install bitarray

In [26]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from bitarray import bitarray
import time

**Loading Dataset**

In [57]:
from google.colab import drive
drive.mount('/content/drive')
path = "drive/MyDrive/mushroom.txt"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
df = pd.read_csv(path,header=None)

In [59]:
print(df)

                                                      0
0     1 5 12 21 23 25 36 39 42 53 56 57 67 71 79 88 ...
1     1 5 12 21 23 25 36 39 42 53 56 57 67 71 79 88 ...
2     1 5 12 21 23 25 36 39 42 50 56 57 67 71 79 88 ...
3     1 5 12 21 23 25 36 39 42 50 56 57 67 71 79 88 ...
4     1 5 12 21 23 25 36 39 42 44 56 57 67 71 79 88 ...
...                                                 ...
8411  1 7 12 13 24 31 34 38 41 44 55 63 67 71 76 85 ...
8412  1 7 12 13 24 31 34 38 41 44 55 63 67 71 76 85 ...
8413  1 7 12 13 24 31 34 38 41 44 55 63 67 71 76 85 ...
8414  1 7 12 13 24 31 34 38 41 44 55 63 67 71 76 85 ...
8415  1 7 12 13 24 31 34 38 41 44 55 63 67 71 76 85 ...

[8416 rows x 1 columns]


**Data Preprocessing**

In [60]:
df.rename(columns={df.columns[0]: 'new'},inplace=True)

In [61]:
df

,new
0,1 5 12 21 23 25 36 39 42 53 56 57 67 71 79 88 ...
1,1 5 12 21 23 25 36 39 42 53 56 57 67 71 79 88 ...
2,1 5 12 21 23 25 36 39 42 50 56 57 67 71 79 88 ...
3,1 5 12 21 23 25 36 39 42 50 56 57 67 71 79 88 ...
4,1 5 12 21 23 25 36 39 42 44 56 57 67 71 79 88 ...
...,...
8411,1 7 12 13 24 31 34 38 41 44 55 63 67 71 76 85 ...
8412,1 7 12 13 24 31 34 38 41 44 55 63 67 71 76 85 ...
8413,1 7 12 13 24 31 34 38 41 44 55 63 67 71 76 85 ...
8414,1 7 12 13 24 31 34 38 41 44 55 63 67 71 76 85 ...


**Total Number of Transactions**

In [62]:
len(df)

8416

**Total number of distinct items**

In [63]:
t1=time.time()
s=set()
database_dict={}
c=0
for i in df.iterrows():
    for j in i[1][0].split(" "):
      if(j!=''):
        if(j not in s):
          s.add(j)
          database_dict[j]=bitarray(len(df))
          database_dict[j].setall(0)
          database_dict[j][c]=1
        else:
          database_dict[j][c]=1
    c=c+1
print(len(s))

119


In [34]:
print(len(database_dict))

119


In [36]:
min_support_count=5050

In [64]:
column_names=["items","support_count","pattern_size"]
df2 = pd.DataFrame(columns = column_names)

In [38]:
one_sorted=sorted([int(i) for i in database_dict])

In [40]:
for i in one_sorted:
  print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
36
38
39
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
60
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
92
93
94
95
96
97
98
100
101
102
103
104
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128


In [65]:
from collections import OrderedDict
diction=OrderedDict()

In [66]:
column_names=["items","support_count","pattern_size"]
df3 = pd.DataFrame(columns = column_names)

In [67]:
row_start=0
row_end=0
count1=0
count2=0
freq_itemsets=[]
t1=time.time()
s=sorted(list(s))
for i in s:
  count1=count1+1
  count=database_dict[i].count(1)
  if(count>=min_support_count):
    count2=count2+1
    df3 = df3.append({'items':[i],'support_count':count,'pattern_size':int(1)},ignore_index=True)
    freq_itemsets.append([[i],count,int(1)])
    row_end=row_end+1
print(count1,count2)
first=row_start
last=0
 

119 8


In [68]:
df3

,items,support_count,pattern_size
0,[36],8200,1
1,[38],6824,1
2,[41],5880,1
3,[67],5316,1
4,[71],5076,1
5,[90],8416,1
6,[94],8216,1
7,[97],7768,1


In [69]:
def check_all_subsets(temp1,min_support_count):
  n=int(math.pow(2,len(temp1)))
  flag=0
  for i in range(n):
    support_subpattern=bitarray(len(df))
    support_subpattern.setall(1)
    for j in range(len(temp1)):
      if i & (1 << j):
        support_subpattern&database_dict[temp1[j]]
    if(support_subpattern.count(1)<min_support_count):
      flag=1
  if(flag==0):
    return 1
  else:
    return 0

In [70]:
df2 = pd.DataFrame(columns = column_names)
rep=0
while(len(freq_itemsets)!=0):
  first=0
  rep=rep+1
  last=len(freq_itemsets)-1
  not_freq_indices=set()
  second_set=[]
  for i in range(first,last+1,1):
    for j in range(i+1,last+1,1):
      c=freq_itemsets[i][0]
      d=freq_itemsets[j][0]
      
      '''if(len(c)==1):
        item_set=[c[0],d[0]]
        flag=0
        support_count=(database_dict[item_set[0]]&database_dict[item_set[1]]).count(1)
        for k in item_set:
          if(database_dict[k].count(1)>=min_support_count):
            pass
          else:
            flag=1
        
        if(database_dict[item_set[0]].count(1)==support_count):
          not_freq_indices.add(i)
        if(database_dict[item_set[1]].count(1)==support_count):
          not_freq_indices.add(j)

        if(flag==0 and support_count>=min_support_count):
          row_end=row_end+1
          second_set.append([item_set,support_count,int(len(item_set))])'''
      if(rep==1):
        item_set=[c[0],d[0]]
        flag=0
        support_count=(database_dict[item_set[0]]&database_dict[item_set[1]]).count(1)
        if(support_count>=min_support_count and database_dict[item_set[0]].count(1)>=min_support_count and database_dict[item_set[1]].count(1)>=min_support_count):
          df3 = df3.append({'items':item_set,'support_count':support_count,'pattern_size':int(len(item_set))},ignore_index=True)
          second_set.append([item_set,support_count,int(len(item_set))])
        if(database_dict[item_set[0]].count(1)==support_count):
          not_freq_indices.add(i)
        if(database_dict[item_set[1]].count(1)==support_count):
          not_freq_indices.add(j)
      else:
        temp1=[]
        for p in c:
          temp1.append(p)
        temp2=[]
        for p in d:
          temp2.append(p)
        temp1.pop()
        temp2.pop()
        if(temp1==temp2):
          pattern=[]
          for p in temp1:
            pattern.append(p)
          pattern.append(c[-1])
          pattern.append(d[-1])
          support_count=bitarray(len(df))
          support_count.setall(1)
          for k in range(len(pattern)):
            support_count=support_count&database_dict[pattern[k]]
          support_count=support_count.count(1)
          if(support_count >= min_support_count and check_all_subsets(pattern,min_support_count)):
            df3 = df3.append({'items':pattern,'support_count':support_count,'pattern_size':int(len(pattern))},ignore_index=True)
            second_set.append([pattern,support_count,int(len(pattern))])
          support_count2=bitarray(len(df))
          support_count2.setall(1)
          for k in range(len(d)):
            support_count2=support_count2&database_dict[d[k]]
          support_count2=support_count2.count(1)
          if(support_count2==support_count):
            not_freq_indices.add(j)
          support_count1=bitarray(len(df))
          support_count1.setall(1)
          temp1.pop()
          for k in range(len(c)):
            support_count1=support_count1&database_dict[c[k]]
          support_count1=support_count1.count(1)
          if(support_count1==support_count):
            not_freq_indices.add(i)
          
          

  freq_indices=[i for i in range(len(freq_itemsets))]
  rem_indices=[i for i in freq_indices if i not in not_freq_indices]
  for i in rem_indices:
    df2 = df2.append({'items':freq_itemsets[i][0],'support_count':freq_itemsets[i][1],'pattern_size':freq_itemsets[i][2]},ignore_index=True)
    diction[str(freq_itemsets[i][0])]=[freq_itemsets[i][0],freq_itemsets[i][1],freq_itemsets[i][2]]
  freq_itemsets=[]
  for i in second_set:
    freq_itemsets.append(i)
t2=time.time()

  
  
  
          
  



**Time taken to extract frequent itemsets using apriori**

In [73]:
t2-t1

17.718475341796875

In [55]:
df2

,items,support_count,pattern_size


In [50]:
df3

,items,support_count,pattern_size
0,[36],8200,1
1,[38],6824,1
2,[41],5880,1
3,[67],5316,1
4,[71],5076,1
5,[90],8416,1
6,[94],8216,1
7,[97],7768,1
8,"[36, 38]",6608,2
9,"[36, 41]",5664,2


In [52]:
df2 = pd.DataFrame(columns = column_names)
for i in diction:
  df2 = df2.append({'items':diction[i][0],'support_count':diction[i][1],'pattern_size':diction[i][2]},ignore_index=True)


In [53]:
df2

,items,support_count,pattern_size
0,[90],8416,1
1,"[36, 90]",8200,2
2,"[38, 90]",6824,2
3,"[41, 90]",5880,2
4,"[67, 90]",5316,2
5,"[71, 90]",5076,2
6,"[90, 94]",8216,2
7,"[90, 97]",7768,2
8,"[94, 97]",7568,2
9,"[36, 90, 94]",8192,3


In [ ]:
diction
len(diction)
  

26

In [ ]:
def calc_support(item):
  su=bitarray(len(df))
  su.setall(1)
  for i in item:
    su=su&database_dict[i]
  return su.count(1)

In [ ]:
keys=[]
for i in reversed(diction):
  item=diction[i][0]
  full_support_count=calc_support(item)
  for k in range(len(item)+1):
    for j in range(k):
        if(str(item[j:k]) in diction):
          print("true",str(item[j:k]),type(item[j:k]))
          if(item[j:k]!=item and item[j:k]!=[] and calc_support(item[j:k])==full_support_count):
            keys.append(str(item[j:k]))
for j in keys:
  del diction[j]
t2=time.time()

true ['38', '90'] <class 'list'>
true ['90'] <class 'list'>
true ['36', '38', '90', '94'] <class 'list'>
true ['38', '90', '94'] <class 'list'>
true ['90', '94'] <class 'list'>
true ['36', '38', '90', '94', '97'] <class 'list'>
true ['36', '90'] <class 'list'>
true ['90'] <class 'list'>
true ['36', '90', '94'] <class 'list'>
true ['90', '94'] <class 'list'>
true ['36', '90', '94', '97'] <class 'list'>
true ['67', '90'] <class 'list'>
true ['90'] <class 'list'>
true ['36', '67', '90', '94'] <class 'list'>
true ['90', '94'] <class 'list'>
true ['41', '90'] <class 'list'>
true ['90'] <class 'list'>
true ['36', '41', '90', '94'] <class 'list'>
true ['41', '90', '94'] <class 'list'>
true ['90', '94'] <class 'list'>
true ['36', '38', '94', '97'] <class 'list'>
true ['38', '90'] <class 'list'>
true ['90'] <class 'list'>
true ['36', '38', '90', '94'] <class 'list'>
true ['38', '90', '94'] <class 'list'>
true ['90', '94'] <class 'list'>
true ['41', '90'] <class 'list'>
true ['90'] <class 'list'

In [ ]:
t2-t1

65.29895305633545

In [ ]:
diction

OrderedDict([("['90']", [['90'], 8416, 1]),
             ("['36', '90']", [['36', '90'], 8200, 2]),
             ("['38', '90']", [['38', '90'], 6824, 2]),
             ("['41', '90']", [['41', '90'], 5880, 2]),
             ("['67', '90']", [['67', '90'], 5316, 2]),
             ("['71', '90']", [['71', '90'], 5076, 2]),
             ("['90', '94']", [['90', '94'], 8216, 2]),
             ("['90', '97']", [['90', '97'], 7768, 2]),
             ("['36', '90', '94']", [['36', '90', '94'], 8192, 3]),
             ("['36', '90', '97']", [['36', '90', '97'], 7576, 3]),
             ("['36', '94', '97']", [['36', '94', '97'], 7568, 3]),
             ("['38', '90', '94']", [['38', '90', '94'], 6632, 3]),
             ("['38', '90', '97']", [['38', '90', '97'], 6464, 3]),
             ("['41', '90', '94']", [['41', '90', '94'], 5688, 3]),
             ("['41', '90', '97']", [['41', '90', '97'], 5232, 3]),
             ("['36', '38', '90', '94']", [['36', '38', '90', '94'], 6608, 4]),
        

In [ ]:
from collections import OrderedDict
diction1=OrderedDict()

In [ ]:
itemsets=[[i] for i in database_dict]

In [ ]:
hash_dictionary={}
for i in range(3011):
  hash_dictionary[i]=[0,[]]

**Hashing**

In [ ]:
def get_support_count_from_database(item):
  support=0
  for index,rows in df.iterrows():
    c=rows.new.split(" ")
    if(item in c):
      print("yes")
      ind=c.index(item)
      for j in range(ind+1,len(c)-1):
        new_candidate=[item,c[j]]
        s1=0
        for k in item:
          s1=s1+ord(k)
        s2=0
        for j in item:
          s2=s2+ord(j)
        hash_value=(s2*10+s2)%3011
        hash_dictionary[hash_value][0]=hash_dictionary[hash_value][0]+1
        hash_dictionary[hash_value][1].append(new_candidate)
      support=support+1
  return support

**Hash Based Technique**

In [ ]:
length_one_patterns=[]
for i in itemsets:
  support=get_support_count_from_database(i[0])
  if(support>=min_support_count):
    length_one_patterns.append(i)

Streaming output truncated to the last 5000 lines.
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
y

In [ ]:
print(length_one_patterns)

[['36'], ['67'], ['71'], ['90'], ['94'], ['97'], ['38'], ['41']]


In [ ]:
for i in hash_dictionary:
  print(hash_dictionary[i])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[0, []]
[34780, [['68', '79'], ['68', '88'], ['68', '90'], ['68', '94'], ['68', '97'], ['68', '100'], ['68', '107'], ['68', '119'], ['68', '122'], ['68', '79'], ['68', '88'], ['68', '90'], ['68', '94'], ['68', '97'], ['68', '100'], ['68', '107'], ['68', '116'], ['68', '122'], ['68', '79'], ['68', '88'], ['68', '90'], ['68', '94'], ['68', '97'], ['68', '100'], ['68', '108'], ['68', '119'], ['68', '122'], ['68', '79'], ['68', '88'], ['68', '90'], ['68', '94'], ['68', '97'], ['68', '100'], ['68', '108'], ['68', '116'], ['68', '122'], ['68', '79'], ['68', '88'], ['68', '90'], ['68', '94'], ['68', '97'], ['68', '100'], ['68', '107'], ['68', '119'], ['68', '122'], ['68', '79'], ['68', '88'], ['68', '90'], ['68', '94'], ['68', '97'], ['68', '100'], ['68', '107'], ['68', '116'], ['68', '122'], ['68', '79'], ['68', '88'], ['68', '90'], ['68', '94'], ['68', '97'], ['68', '100'], ['68', '108'], ['68', '119'], ['68', '122'], [

In [ ]:
s=0
c=[]
for i in hash_dictionary:
  if(hash_dictionary[i][0]>=min_support_count):
    c.append([list(j) for j in set(tuple(j) for j in hash_dictionary[i][1])])


In [ ]:
s=[]
for i in c:
  for j in i:
    if(j not in s):
      s.append(j)

In [ ]:
print(s)

[['1', '18'], ['1', '87'], ['1', '60'], ['1', '119'], ['1', '84'], ['1', '16'], ['1', '107'], ['1', '23'], ['1', '36'], ['1', '9'], ['1', '67'], ['1', '64'], ['1', '112'], ['1', '25'], ['1', '98'], ['1', '7'], ['1', '56'], ['1', '41'], ['1', '70'], ['1', '49'], ['1', '72'], ['1', '46'], ['1', '127'], ['1', '8'], ['1', '77'], ['1', '85'], ['1', '43'], ['1', '62'], ['1', '5'], ['1', '90'], ['1', '57'], ['1', '100'], ['1', '44'], ['1', '92'], ['1', '55'], ['1', '53'], ['1', '12'], ['1', '68'], ['1', '58'], ['1', '121'], ['1', '120'], ['1', '118'], ['1', '20'], ['1', '65'], ['1', '17'], ['1', '24'], ['1', '117'], ['1', '115'], ['1', '71'], ['1', '38'], ['1', '3'], ['1', '101'], ['1', '69'], ['1', '128'], ['1', '123'], ['1', '22'], ['1', '6'], ['1', '13'], ['1', '75'], ['1', '47'], ['1', '11'], ['1', '63'], ['1', '114'], ['1', '66'], ['1', '34'], ['1', '15'], ['1', '54'], ['1', '97'], ['1', '88'], ['1', '116'], ['1', '108'], ['1', '76'], ['1', '26'], ['1', '94'], ['1', '51'], ['1', '50'], [

In [ ]:
freq_itemsets=[]
for i in s:
  sup=(database_dict[i[0]]&database_dict[i[1]]).count(1)
  if((database_dict[i[0]]&database_dict[i[1]]).count(1)>=min_support_count and database_dict[i[0]].count(1)>=min_support_count and database_dict[i[1]].count(1)>=min_support_count):
    freq_itemsets.append([i,s,int(2)])
rep=2
while(len(freq_itemsets)!=0):
  first=0
  rep=rep+1
  last=len(freq_itemsets)-1
  not_freq_indices=set()
  second_set=[]
  for i in range(first,last+1,1):
    for j in range(i+1,last+1,1):
      c=freq_itemsets[i][0]
      d=freq_itemsets[j][0]
      
      
      if(rep==1):
        item_set=[c[0],d[0]]
        flag=0
        support_count=(database_dict[item_set[0]]&database_dict[item_set[1]]).count(1)
        if(support_count>=min_support_count and database_dict[item_set[0]].count(1)>=min_support_count and database_dict[item_set[1]].count(1)>=min_support_count):
          second_set.append([item_set,support_count,int(len(item_set))])
        if(database_dict[item_set[0]].count(1)==support_count):
          not_freq_indices.add(i)
        if(database_dict[item_set[1]].count(1)==support_count):
          not_freq_indices.add(j)
      else:
        temp1=[]
        for p in c:
          temp1.append(p)
        temp2=[]
        for p in d:
          temp2.append(p)
        temp1.pop()
        temp2.pop()
        if(temp1==temp2):
          pattern=[]
          for p in temp1:
            pattern.append(p)
          pattern.append(c[-1])
          pattern.append(d[-1])
          support_count=bitarray(len(df))
          support_count.setall(1)
          for k in range(len(pattern)):
            support_count=support_count&database_dict[pattern[k]]
          support_count=support_count.count(1)
          if(support_count >= min_support_count and check_all_subsets(pattern,min_support_count)):
            second_set.append([pattern,support_count,int(len(pattern))])
          support_count2=bitarray(len(df))
          support_count2.setall(1)
          for k in range(len(d)):
            support_count2=support_count2&database_dict[d[k]]
          support_count2=support_count2.count(1)
          if(support_count2==support_count):
            not_freq_indices.add(j)
          support_count1=bitarray(len(df))
          support_count1.setall(1)
          temp1.pop()
          for k in range(len(c)):
            support_count1=support_count1&database_dict[c[k]]
          support_count1=support_count1.count(1)
          if(support_count1==support_count):
            not_freq_indices.add(i)
          
          

  freq_indices=[i for i in range(len(freq_itemsets))]
  rem_indices=[i for i in freq_indices if i not in not_freq_indices]
  for i in rem_indices:
    df2 = df2.append({'items':freq_itemsets[i][0],'support_count':freq_itemsets[i][1],'pattern_size':freq_itemsets[i][2]},ignore_index=True)
    diction1[str(freq_itemsets[i][0])]=[freq_itemsets[i][0],freq_itemsets[i][1],freq_itemsets[i][2]]
  freq_itemsets=[]
  for i in second_set:
    freq_itemsets.append(i)

In [ ]:
for i in diction1:
  print(i)

['41', '90']
['71', '90']
['90', '97']
['90', '94']
['36', '90']
['38', '90']
['94', '97']
['67', '90']
['41', '97', '90']
['41', '90', '94']
['36', '94', '90']
['36', '97', '90']
['90', '97', '94']
['36', '67', '90']
['38', '97', '90']
['38', '90', '94']
['67', '90', '94']
['36', '41', '94', '90']
['36', '38', '94', '90']
['36', '38', '97', '90']
['36', '94', '97', '90']
['36', '94', '67', '90']
['38', '97', '90', '94']
['36', '38', '94', '97', '90']


In [ ]:
print(len(diction1))

24


**Partitioning**

In [74]:
len(df)

8416

**Here considering dividing into 6 number of parts**

In [76]:
num_items_in_each_part=int(len(df)/6)

In [84]:
df_parts=[[],[],[],[],[],[]]
count=0
present=0
for i in df.iterrows():
  if(count<num_items_in_each_part):
    df_parts[present].append(i[1][0].split(" "))
    count=count+1
  elif(count==num_items_in_each_part):
    present=present+1
    if(present==len(df_parts)):
      present=present-1
    df_parts[present].append(i[1][0].split(" "))
    count=1



In [124]:
print(len(df_parts[0]))
print(len(df_parts[1]))
print(len(df_parts[2]))
print(len(df_parts[3]))
print(len(df_parts[4]))
print(len(df_parts[5]))

1402
1402
1402
1402
1402
1406


In [125]:
support_count_parts=math.ceil(min_support_count/6)

In [126]:
print(df_parts[0])

[['1', '5', '12', '21', '23', '25', '36', '39', '42', '53', '56', '57', '67', '71', '79', '88', '90', '94', '97', '104', '113', '120', '128', ''], ['1', '5', '12', '21', '23', '25', '36', '39', '42', '53', '56', '57', '67', '71', '79', '88', '90', '94', '97', '104', '108', '120', '128', ''], ['1', '5', '12', '21', '23', '25', '36', '39', '42', '50', '56', '57', '67', '71', '79', '88', '90', '94', '97', '104', '113', '120', '128', ''], ['1', '5', '12', '21', '23', '25', '36', '39', '42', '50', '56', '57', '67', '71', '79', '88', '90', '94', '97', '104', '108', '120', '128', ''], ['1', '5', '12', '21', '23', '25', '36', '39', '42', '44', '56', '57', '67', '71', '79', '88', '90', '94', '97', '104', '113', '120', '128', ''], ['1', '5', '12', '21', '23', '25', '36', '39', '42', '44', '56', '57', '67', '71', '79', '88', '90', '94', '97', '104', '108', '120', '128', ''], ['1', '5', '12', '21', '23', '26', '36', '39', '42', '53', '56', '57', '67', '71', '79', '88', '90', '94', '97', '104', '11

In [139]:
df_dict_0={}
df_dict_1={}
df_dict_2={}
df_dict_3={}
df_dict_4={}
df_dict_5={}

In [173]:
def partition_count(freq_itemsets,dataframe,dic,sup_parts):
  rep=0
  dataframe= pd.DataFrame(columns = column_names)
  while(len(freq_itemsets)!=0):
    first=0
    rep=rep+1
    last=len(freq_itemsets)-1
    second_set=[]
    for i in range(first,last+1,1):
      for j in range(i+1,last+1,1):
        c=freq_itemsets[i][0]
        d=freq_itemsets[j][0]
        if(len(c)==0 or len(d)==0):
          print("yes")
        
        '''if(len(c)==1):
          item_set=[c[0],d[0]]
          flag=0
          support_count=(database_dict[item_set[0]]&database_dict[item_set[1]]).count(1)
          for k in item_set:
            if(database_dict[k].count(1)>=min_support_count):
              pass
            else:
              flag=1
          
          if(database_dict[item_set[0]].count(1)==support_count):
            not_freq_indices.add(i)
          if(database_dict[item_set[1]].count(1)==support_count):
            not_freq_indices.add(j)

          if(flag==0 and support_count>=min_support_count):
            row_end=row_end+1
            second_set.append([item_set,support_count,int(len(item_set))])'''
        if(rep==1):
          item_set=[c[0],d[0]]
          flag=0
          support_count=(dic[item_set[0]]&dic[item_set[1]]).count(1)
          if(support_count>=sup_parts and dic[item_set[0]].count(1)>=sup_parts and dic[item_set[1]].count(1)>=sup_parts):
            dataframe.append({'items':item_set,'support_count':support_count,'pattern_size':int(len(item_set))},ignore_index=True)
            second_set.append([item_set,support_count,int(len(item_set))])
        else:
          temp1=[]
          for p in c:
            temp1.append(p)
          temp2=[]
          for p in d:
            temp2.append(p)
          temp1.pop()
          temp2.pop()
          if(temp1==temp2):
            pattern=[]
            for p in temp1:
              pattern.append(p)
            pattern.append(c[-1])
            pattern.append(d[-1])
            support_count=bitarray(len(df))
            support_count.setall(1)
            for k in range(len(pattern)):
              support_count=support_count&dic[pattern[k]]
            support_count=support_count.count(1)
            if(support_count >= sup_parts and check_all_subsets(pattern,sup_parts)):
              dataframe.append({'items':pattern,'support_count':support_count,'pattern_size':int(len(pattern))},ignore_index=True)
              second_set.append([pattern,support_count,int(len(pattern))])
            
            
            

    
    for i in second_set:
      freq_itemsets.append(i)
  t2=time.time()
  return dataframe

In [171]:
s0=set()
c=0
for i in df_parts[0]:
    for j in i:
      if(j in s0):
        df_dict_0[j][c]=1
      else:
        if(j!=''):
          s0.add(j)
          df_dict_0[j]=bitarray(len(df))
          df_dict_0[j].setall(0)
          df_dict_0[j][c]=1
    c=c+1


In [154]:
print(len(df_dict_0))

62


In [168]:
print(s0)

{'79', '107', '8', '97', '46', '6', '31', '122', '44', '11', '56', '13', '41', '39', '125', '21', '108', '121', '5', '1', '100', '23', '116', '38', '24', '50', '53', '60', '104', '42', '128', '119', '113', '25', '120', '9', '118', '58', '71', '47', '126', '3', '22', '26', '64', '12', '43', '2', '68', '90', '57', '69', '55', '124', '32', '88', '36', '94', '62', '67', '16'}


In [176]:
freq_itemsets=[]
for i in s0:
  sup=df_dict_0[i].count(1)
  if(df_dict_0[i].count(1)>=support_count_parts):
    freq_itemsets.append([[i],sup,int(1)])
print(freq_itemsets)
df_part_0 = pd.DataFrame(columns = column_names)
df_part_0=partition_count(freq_itemsets,df_part_0,df_dict_0,support_count_parts)



Streaming output truncated to the last 5000 lines.
['71', '67'] ['97', '38', '55']
['71', '67'] ['97', '38', '88']
['71', '67'] ['97', '38', '36']
['71', '67'] ['97', '38', '94']
['71', '67'] ['97', '38', '67']
['71', '67'] ['97', '38', '23']
['71', '67'] ['97', '38', '38']
['71', '67'] ['97', '38', '104']
['71', '67'] ['97', '38', '71']
['71', '67'] ['97', '38', '90']
['71', '67'] ['97', '38', '55']
['71', '67'] ['97', '38', '88']
['71', '67'] ['97', '38', '36']
['71', '67'] ['97', '38', '94']
['71', '67'] ['97', '38', '67']
['71', '67'] ['97', '104', '71']
['71', '67'] ['97', '104', '90']
['71', '67'] ['97', '104', '55']
['71', '67'] ['97', '104', '88']
['71', '67'] ['97', '104', '36']
['71', '67'] ['97', '104', '94']
['71', '67'] ['97', '104', '67']
['71', '67'] ['97', '104', '1']
['71', '67'] ['97', '104', '23']
['71', '67'] ['97', '104', '38']
['71', '67'] ['97', '104', '104']
['71', '67'] ['97', '104', '71']
['71', '67'] ['97', '104', '90']
['71', '67'] ['97', '104', '55']
['71',

KeyboardInterrupt: ignored

In [119]:
print(df_part_0)

Empty DataFrame
Columns: [items, support_count, pattern_size]
Index: []


In [92]:
s1=set()
c=0
for i in df_parts[1]:
    for j in i:
      if(j!=''):
        if(j not in s):
          s.add(j)
          df_dicts[1]=bitarray(len(df))
          df_dicts[1].setall(0)
          df_dicts[1][c]=1
        else:
          df_dicts[1][c]=1
    c=c+1


In [93]:
s2=set()
c=0
for i in df_parts[2]:
    for j in i:
      if(j!=''):
        if(j not in s):
          s.add(j)
          df_dicts[2]=bitarray(len(df))
          df_dicts[2].setall(0)
          df_dicts[2][c]=1
        else:
          df_dicts[2][c]=1
    c=c+1


In [94]:
s3=set()
c=0
for i in df_parts[3]:
    for j in i:
      if(j!=''):
        if(j not in s):
          s.add(j)
          df_dicts[3]=bitarray(len(df))
          df_dicts[3].setall(0)
          df_dicts[3][c]=1
        else:
          df_dicts[3][c]=1
    c=c+1


In [97]:
s4=set()
c=0
for i in df_parts[4]:
    for j in i:
      if(j!=''):
        if(j not in s):
          s.add(j)
          df_dicts[4]=bitarray(len(df))
          df_dicts[4].setall(0)
          df_dicts[4][c]=1
        else:
          df_dicts[4][c]=1
    c=c+1


In [98]:
s5=set()
c=0
for i in df_parts[5]:
    for j in i:
      if(j!=''):
        if(j not in s):
          s.add(j)
          df_dicts[5]=bitarray(len(df))
          df_dicts[5].setall(0)
          df_dicts[5][c]=1
        else:
          df_dicts[5][c]=1
    c=c+1
